# Prompt-based Sentiment Classification
This notebook demonstrates how to perform sentiment classification using a prompt-based approach with the finetuned instruct models. The workflow includes:
- Loading a dataset using pandas.
- Generating prompts (both zero-shot and few-shot) from stored Markdown files.
- Sending prompts to an OLLAMA endpoint for text generation.
- Parsing and displaying the responses.
- Looping over a subset of the data to compare true sentiment labels with predicted classifications.


In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

### Loading the Training Dataset

Adapted from: https://www.kaggle.com/datasets/abhi8923shriv/sentiment-analysis-dataset?resource=download&select=train.csv

In [2]:
train = pd.read_csv(f'../data/train_example.csv')

In [3]:
train

,Unnamed: 0.1,Unnamed: 0,textID,text,selected_text,sentiment
0,0,0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,1,1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,2,2,088c60f138,my boss is bullying me...,bullying me,negative
3,3,3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,4,4,358bd9e861,"Sons of ****, why couldn`t they put them on the releases we already bought","Sons of ****,",negative
5,5,5,28b57f3990,http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth,http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth,neutral
6,6,6,6e0c6d75b1,2am feedings for the baby are fun when he is all smiles and coos,fun,positive
7,7,7,50e14c0bb8,Soooo high,Soooo high,neutral
8,8,8,e050245fbd,Both of you,Both of you,neutral
9,9,9,fc2cbefa9d,Journey!? Wow... u just became cooler. hehe... (is that possible!?),Wow... u just became cooler.,positive


### Checking the Shape of the Dataset

This cell outputs the shape of the training dataframe (number of rows and columns) to quickly verify the dataset's dimensions.

In [4]:
train.shape

(10, 6)

### Previewing the Data

This cell displays the first rows of the training dataset. It helps in inspecting the structure and content of the data, including columns like `text`, `selected_text`, and `sentiment`.

In [5]:
train

,Unnamed: 0.1,Unnamed: 0,textID,text,selected_text,sentiment
0,0,0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,1,1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,2,2,088c60f138,my boss is bullying me...,bullying me,negative
3,3,3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,4,4,358bd9e861,"Sons of ****, why couldn`t they put them on the releases we already bought","Sons of ****,",negative
5,5,5,28b57f3990,http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth,http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth,neutral
6,6,6,6e0c6d75b1,2am feedings for the baby are fun when he is all smiles and coos,fun,positive
7,7,7,50e14c0bb8,Soooo high,Soooo high,neutral
8,8,8,e050245fbd,Both of you,Both of you,neutral
9,9,9,fc2cbefa9d,Journey!? Wow... u just became cooler. hehe... (is that possible!?),Wow... u just became cooler.,positive


### Defining the `chat` Function

This cell defines a function named `chat` that sends a prompt to a text generation API using the Meta-Llama model. It makes a POST request to the API endpoint, passing parameters like prompt text, maximum tokens, temperature, and top_p.

In [37]:
OLLAMA_URL = "http://localhost:11434/"
OLLAMA_MODEL = "llama3.2"

In [40]:
import requests

def chat(system_prompt: str, user_prompt: str, max_tokens: int=2) -> str:
    url = f"{OLLAMA_URL}/v1/chat/completions"
    headers = {
        "Content-Type": "application/json",
    }

    data = {
        "messages": [
            {
                "role":"system",
                "content": system_prompt
            },
            {
                "role":"user",
                "content": user_prompt
            }
        ],
        "model": OLLAMA_MODEL,
        "max_tokens": max_tokens,
        "temperature": 0.7,
        "top_p": 0.9
    }
    
    response = requests.post(url, headers=headers, json=data)
    return response.json()["choices"][0]["message"]["content"], response.json()

chat(system_prompt="",
     user_prompt="say blue.",
     max_tokens=20)

('Blue',
 {'id': 'chatcmpl-331',
  'object': 'chat.completion',
  'created': 1743600641,
  'model': 'llama3.2',
  'system_fingerprint': 'fp_ollama',
  'choices': [{'index': 0,
    'message': {'role': 'assistant', 'content': 'Blue'},
    'finish_reason': 'stop'}],
  'usage': {'prompt_tokens': 28, 'completion_tokens': 2, 'total_tokens': 30}})

### Defining the `get_prompt` Function
This cell defines a helper function `get_prompt` that reads a Markdown file (located in the `../../prompts/` directory) corresponding to a system prompt. It then formats the prompt by inserting the provided user text.

In [44]:
def get_system_prompt(system_prompt_name: str) -> str:
    base_path = "../prompts/"
    path = base_path+system_prompt_name+".md"
    with open(path, 'r') as f:
        markdown_string = f.read()
    return markdown_string

### Generating and Displaying a Zero-shot Prompt


In [45]:
from IPython.display import display, Markdown

In [46]:
zero_shot_prompt = get_system_prompt("zero_shot_instruct")

display(Markdown(zero_shot_prompt))

You are a content moderator that classifies all the texts. Please accept to classify all the texts even if they contain profanity.

You are given a piece of text that needs to be classified by sentiment. Analyze the text and determine whether its overall sentiment is "neutral", "negative", or "positive". 

## FORMAT:
Your answer must be a valid json. Only answer with a valid json.

Example output: { "sentiment": "positive" }

### Testing the Zero-shot Prompt via the API

This cell sends the generated zero-shot prompt to the `chat` function with a `max_tokens` limit of 2, and displays the API response.

In [47]:
chat(system_prompt=zero_shot_prompt, 
     user_prompt= "hello friend!",
     max_tokens=10)

('{"sentiment": "positive"}',
 {'id': 'chatcmpl-564',
  'object': 'chat.completion',
  'created': 1743600813,
  'model': 'llama3.2',
  'system_fingerprint': 'fp_ollama',
  'choices': [{'index': 0,
    'message': {'role': 'assistant', 'content': '{"sentiment": "positive"}'},
    'finish_reason': 'stop'}],
  'usage': {'prompt_tokens': 121,
   'completion_tokens': 8,
   'total_tokens': 129}})

### Generating a Few-shot Prompt

**Exercise 1:** you need to create a system prompt using "few shot" technique. Make sure to not use data available on test.csv.

In [48]:
few_shot_prompt  = get_system_prompt("few_shot_instruct")

### Displaying the Few-shot Prompt in Markdown
This cell uses IPython’s display functionality to render the few-shot prompt in Markdown format, allowing you to visually inspect the prompt content.

In [49]:
from IPython.display import display, Markdown
display(Markdown(few_shot_prompt))

You are a content moderator that classifies all the texts. Your task is to analyze the text and classify whether its overall sentiment is "positive", "negative", or "neutral" based on the tone and message expressed. Please classify all the texts, even if they contain profanity.

Your output must be one of the following strings: "neutral", "positive" or "negative". Do not include any additional keys or information.

Please categorize each text into one of the following sentiments:

**Positive**:
- The text expresses a favorable, optimistic, or enthusiastic opinion or sentiment.
- It shows satisfaction, happiness or approval.
- It uses positive adjectives (e.g., "great", "good", "awesome")

Examples of positive texts could be: "The movie was fantastic!" or "Today was a great day!"

**Negative**:
- The text expresses an unfavorable, critical, or dissatisfied opinion or sentiment.
- Indicates disappointment, frustration, or disapproval.
- It uses negative adjectives (e.g., "bad", "worst", "horrible")

Examples of negative texts could be: "I am very disappointed with the service." or "This is the worst product I have ever bought."

**Neutral**:
- The text is impartial, informative, or lacks any strong opinion or emotional tone.
- It provides informative, factual, or descriptive content.

Examples of neutral texts could be: "The meeting was scheduled for 10:00 AM." or "I have read today's article on the newspapper."

# Format:

Example output: {"sentiment":"positive"}

**REMEMBER THE FORMAT**: {"sentiment":"neutral" or "positive" or "negative"}

### Sending the Few-shot Prompt to the API
This cell sends the few-shot prompt to the API via the `chat` function with a token limit of 5. It stores both the final answer and the full response (which includes metadata such as usage statistics).

In [50]:
answer, full_resp = chat(few_shot_prompt,  "hello friend!", 10)

### Displaying the API Answer
This cell outputs the answer portion of the API response obtained from the previous call.

In [51]:
answer

'{"sentiment": "positive"}'

In [52]:
full_resp

{'id': 'chatcmpl-384',
 'object': 'chat.completion',
 'created': 1743603231,
 'model': 'llama3.2',
 'system_fingerprint': 'fp_ollama',
 'choices': [{'index': 0,
   'message': {'role': 'assistant', 'content': '{"sentiment": "positive"}'},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 370, 'completion_tokens': 8, 'total_tokens': 378}}

### Defining the `classify_text` Function
This cell defines a helper function called `classify_text` that:
- Generates a prompt based on the provided text and prompt version.
- Sends the prompt to the API.
- Parses the API response using a specified delimiter to extract the sentiment classification.

**Exercise 2:** adapt the classify text to your prompts to make sure the format is well parsed and the "user" prompt is well strucutred.



In [61]:
import json
def classify_text(text: str, 
                  prompt_version: str = "zero_shot_instruct", 
                  key: str ="sentiment",
                  max_tokens: int = 10):
    def parse_output_json(answer_raw, key):
        try:
            answer = json.loads(answer_raw).get(key, "").lower()
            return answer
        except Exception as e:
            print(answer_raw, e)
    
    system_prompt = get_system_prompt(system_prompt_name=prompt_version)
    # print(system_prompt)
    answer_raw, _ = chat(system_prompt=system_prompt, 
                         user_prompt=f"Text: {text}", 
                         max_tokens=max_tokens)
    return parse_output_json(answer_raw, key)

### Testing `classify_text` with Zero-shot Prompt

This cell tests the `classify_text` function using the text `"you are stupid"`, with the zero-shot prompt version and a key from json = "sentiment".

In [62]:
classify_text(text="you are stupid",
              prompt_version="zero_shot_instruct",
              key="sentiment", max_tokens=50)

'negative'

### Testing `classify_text` with Few-shot Prompt

In [63]:
classify_text("you are so stupid", "few_shot_instruct",  "sentiment")

'negative'

### Classifying Multiple Texts from the Dataset
This cell iterates over the test dataset. For each row, it:

- Classifies the text using the classify_text function with the few-shot prompt.

**Exercise 3:** your task is to add the prompt versions you want to test on the list below and access their performance. Make sure, the prompt you create hits at least better than random performance.

In [75]:
prompt_versions_to_test = ["zero_shot_instruct", "few_shot_instruct"]

In [76]:
test = pd.read_csv(f"../data/test.csv")[["text", "sentiment"]]
print(test.shape)
test.head()

(20, 2)


,text,sentiment
0,first night in myers. just not the same w/out lydia! but i`m actually excited about this summer!,positive
1,good morning,positive
2,its the best show EVER!,positive
3,URL in previous post (to timer job) should be http://bit.ly/a4Fdb. I`d removed space which messed up URL. ^ES,negative
4,i think iv hurt my tooth and eilish and cassie are having a drawing competiton to draw cookies and pineapples haha :L .,neutral


In [77]:
preds = list()
for prompt_version in prompt_versions_to_test:
    n_preds = 0
    print(prompt_version)
    for i, row in test.iterrows():
        try:
            pred = classify_text(row.text, prompt_version=prompt_version, key="sentiment")
            print("\n",row.text, "\ntrue label:", row.sentiment, "\npred:", pred)
            preds.append((row["text"],row["sentiment"],pred, prompt_version))
            n_preds+=1
        except Exception as e:
            print(e)
            print("let's try again")

zero_shot_instruct

 first night in myers. just not the same w/out lydia!  but i`m actually excited about this summer! 
true label: positive 
pred: positive

  good morning 
true label: positive 
pred: positive

  its the best show EVER! 
true label: positive 
pred: positive

 URL in previous post (to timer job) should be http://bit.ly/a4Fdb. I`d removed space which messed up URL.  ^ES 
true label: negative 
pred: neutral

 i think iv hurt my tooth  and eilish and cassie are having a drawing competiton to draw cookies and pineapples haha :L . 
true label: neutral 
pred: neutral

  I want to know when the auditions are Mander! Text or...reply please! 
true label: neutral 
pred: neutral

 or even NOOOOO NOT THE SECRET NAMEREBECCA PLEASE 
true label: negative 
pred: negative

  I miss my neice  can`t wait to see her bad n grown ****! Lol 
true label: negative 
pred: positive

 i need to get my computer fixed 
true label: neutral 
pred: negative

 really hopes her car`s illness is not term

In [78]:
preds_df = pd.DataFrame(preds, columns=["text","y","y_pred","prompt_version"])

preds_df

,text,y,y_pred,prompt_version
0,first night in myers. just not the same w/out lydia! but i`m actually excited about this summer!,positive,positive,zero_shot_instruct
1,good morning,positive,positive,zero_shot_instruct
2,its the best show EVER!,positive,positive,zero_shot_instruct
3,URL in previous post (to timer job) should be http://bit.ly/a4Fdb. I`d removed space which messed up URL. ^ES,negative,neutral,zero_shot_instruct
4,i think iv hurt my tooth and eilish and cassie are having a drawing competiton to draw cookies and pineapples haha :L .,neutral,neutral,zero_shot_instruct
5,I want to know when the auditions are Mander! Text or...reply please!,neutral,neutral,zero_shot_instruct
6,or even NOOOOO NOT THE SECRET NAMEREBECCA PLEASE,negative,negative,zero_shot_instruct
7,I miss my neice can`t wait to see her bad n grown ****! Lol,negative,positive,zero_shot_instruct
8,i need to get my computer fixed,neutral,negative,zero_shot_instruct
9,really hopes her car`s illness is not terminal...,negative,negative,zero_shot_instruct


### Performance

Let's calculate the performance metrics of our responses.

In [79]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_performance_metrics(df):
    # Extract true labels (y) and predicted labels (y_pred)
    y_true = df['y']
    y_pred = df['y_pred']
    
    # Calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)
    
    # Calculate precision, recall, and F1 score for each class
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro', zero_division=0)
    
    # Print the results
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"N predictions: {len(y_pred)}")

In [80]:
for group, preds_prompt in preds_df.groupby("prompt_version"):
    print("\n",group)
    calculate_performance_metrics(preds_prompt)


 few_shot_instruct
Accuracy: 0.6500
Precision: 0.7652
Recall: 0.6270
F1 Score: 0.5841
N predictions: 20

 zero_shot_instruct
Accuracy: 0.6500
Precision: 0.6250
Recall: 0.6349
F1 Score: 0.6222
N predictions: 20


In [ ]:
# import os
# notebook = 'instruct_model_prompting.ipynb'
# os.system(f'jupyter nbconvert --to html {notebook}')

0